In [ ]:
import glob
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'  # 也可以使用 tensorflow
# os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,exception_verbosity=high'
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cuda,optimizer=fast_compile'
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import keras.backend as K

K.set_image_data_format('channels_last')

import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, LSTM, Concatenate, Dense, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten, UpSampling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.datasets import mnist
from urllib.request import urlretrieve
from keras.optimizers import RMSprop, SGD, Adam, sgd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from keras.models import model_from_json

# from read_data import *
a = %pwd
print(a)

%run /root/jupyter/inspace/sang-min/myo_proejct/load_data.py import DataLoader
#%load load_data import DataLoader
#from load_data import DataLoader


'''
Model structure

Input : (100) Vector
Output : (64, 64, 1) image

Generator
100 -> 256
256 -> 16 x 16
16 x 16 -> 32 x 32
32 x 32 -> 64 x 64
64 x 64 -> 128 x 128

Discriminator
128 x 128 -> 64 x 64
64 x 64 -> 32 x 32
32 x 32 -> 16 x 16
16 x 16 -> 8 x 8
8 x 8 -> 64
64 -> 32 (16, 10)

'''
loader = DataLoader(data_path='./dataset_2018_05_16/', is_real_image=False)

conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02)

lstm_size = (300, 16)
noise_size = 100
image_size = 128
input_size = 100
image_channel = 1
learning_rate = 2e-4

def load_model():
    json_file = open('./model2_output/load_model/g_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    load_g_model = model_from_json(loaded_model_json)
    # load weights into new model
    load_g_model.load_weights("./model2_output/load_model/g_model.h5")
    
    json_file = open('./model2_output/load_model/d_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    load_d_model = model_from_json(loaded_model_json)
    # load weights into new model
    load_d_model.load_weights("./model2_output/load_model/d_model.h5")
    
    json_file = open('./model2_output/load_model/combined_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    load_combined_model = model_from_json(loaded_model_json)
    # load weights into new model
    load_combined_model.load_weights("./model2_output/load_model/combined_model.h5")
    
    print("Loaded model from disk")
    
    return [load_g_model, load_d_model, load_combined_model]

def sample_generation(num, net_g):
    for _ in range(num):
        gan_image = net_g.predict(noise)
        cv2.imwrite('./model2_output/image/' + 'sample image'+ str(_) + '.png', gan_image[_] * 127.5)
        
    print("generated image")


def generative_model(noise_size, image_channel):
    
    print(" _ : ", noise_input)
    _ = Dense(256, input_shape=(100,), activation='relu', bias_initializer='glorot_normal',
              kernel_initializer='glorot_normal')(noise_input)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Reshape((16, 16, 1), input_shape=(256,))(_)

    _ = Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer=conv_init, input_shape=(16, 16, 1))(
        _)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Activation(activation='relu')(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=256, kernel_size=3, padding='same', kernel_initializer=conv_init,
                        input_shape=(16, 16, 128))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Activation(activation='relu')(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=512, kernel_size=3, padding='same',  kernel_initializer=conv_init,
                        input_shape=(32, 32, 256))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Activation(activation='relu')(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=256, kernel_size=3, padding='same', kernel_initializer=conv_init,
                        input_shape=(64, 64, 512))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Activation(activation='relu')(_)

    #_ = UpSampling2D()(_)
    _ = Conv2D(filters=1, kernel_size=3, padding='same', kernel_initializer=conv_init,
               input_shape=(128, 128, 256))(_)
    _ = Activation(activation='tanh')(_)

    return Model(inputs=noise_input, outputs=_)


def discriminative_model(image_size, image_channel):
    _ = inputs = Input(shape=(image_size, image_size, image_channel))

    _ = Conv2D(filters=256, kernel_size=(2, 2), strides=2, padding='same', input_shape=(128, 128, 1),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=512, kernel_size=(1, 1), strides=2, padding='same', input_shape=(64, 64, 256),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=256, kernel_size=(2, 2), strides=2, padding='same', input_shape=(32, 32, 512),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=128, kernel_size=(2, 2), strides=2, padding='same', input_shape=(16, 16, 256),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=128, kernel_size=(2, 2), strides=2, padding='same', input_shape=(8, 8, 128),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=image_channel, kernel_size=(2, 2), strides=1, padding='same', input_shape=(4, 4, 128),
               kernel_initializer=conv_init)(_)
    # _ = LeakyReLU(alpha=0.2)(_)

    outputs = Flatten()(_)
    outputs = Dense(1, activation='sigmoid')(outputs)

    return Model(inputs=inputs, outputs=outputs)

discriminator_optim = sgd(lr=0.01, momentum=0.9, nesterov=True)
generator_optim = Adam(lr=1e-3)
adam = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)  # as described in appendix A of DeepMind's AC-GAN paper

# lstm_input = Input(shape=lstm_size)
noise_input = Input(shape=(noise_size,))
real_image = Input(shape=(image_size, image_size, image_channel))

'''
net_g = generative_model(noise_size, image_channel)
fake_image = net_g(noise_input)
net_g.summary()

net_d = discriminative_model(image_size, image_channel)
net_d.summary()

combined_output = net_d(fake_image)
combined_model = Model(inputs=[noise_input], outputs=[combined_output], name='combined')
'''
net_g, net_d, combined_model = load_model()

net_g.summary()
net_d.summary()

fake_image = net_g(noise_input)

net_g.compile(loss='binary_crossentropy', optimizer=adam)
net_d.compile(loss='binary_crossentropy', optimizer=adam)
net_d.trainable = False
combined_model.compile(loss='binary_crossentropy', optimizer=adam)

combined_model.summary()

'''
loader = DataLoader(data_path='./MYO_Dataset_label/')

emg = loader.load_emg_data()
image, label = loader.load_image()

print(emg.shape)
print(image.shape, label)

emg = loader.get_emg_datas(10)
images, labels = loader.get_images(10)
print(emg.shape, images.shape, labels.shape)

'''

epoch = 30000
i = 0
time_0 = time.time()
batch_size = 64

d_step = 1
g_step = 2

g_loss_history = []
d_loss_real_history = []
d_loss_fake_history = []

while i <= epoch:
    # x_train = loader.get_emg_datas(batch_size)
    images = loader.get_images(batch_size)
    
    for _ in range(d_step):
        noise = np.random.normal(size=[batch_size, noise_size])

        g_z = net_g.predict(noise)
    
        d_loss_real = net_d.train_on_batch(images, np.random.uniform(low=0.7, high=1.2, size=batch_size))
        d_loss_fake = net_d.train_on_batch(g_z, np.random.uniform(low=0.0, high=0.3, size=batch_size))
    
        d_loss_real_history.append(d_loss_real)
        d_loss_fake_history.append(d_loss_fake)
        d_loss = np.sum([d_loss_fake, d_loss_real])    
        
    for _ in range(g_step):
        noise = np.random.normal(size=[batch_size, noise_size])
        combined_loss = combined_model.train_on_batch(noise, np.random.uniform(low=0.7, high=1.2, size=batch_size))
    
        g_loss_history.append(combined_loss)
    
    print("%d [D loss real: %f] [D loss fake: %f] [D loss: %f] [G loss: %f]" % (i, d_loss_real, d_loss_fake, d_loss, combined_loss))
    
    if i % 500 == 0:
        gan_image = net_g.predict(noise)
        print("gan imaga2 : ", gan_image[0].shape)
        cv2.imwrite('./model2_output/image/' + 'fake_image'+ str(i) + '.png', gan_image[0] * 127.5)
        #cv2.imwrite('./output_image3/' + 'real_image'+ str(i) + '.png', images[0] * 127.5)
        
    i +=1;

sample_generation(32, net_g)

net_g.save_weights("./model2_output/save_model/g_model.h5")
net_d.save_weights("./model2_output/save_model/d_model.h5")
combined_model.save_weights("./model2_output/save_model/combined_model.h5")

g_model_json = net_g.to_json()
with open("./model2_output/save_model/g_model.json", "w") as json_file:
    json_file.write(g_model_json)
    
d_model_json = net_d.to_json()
with open("./model2_output/save_model.json", "w") as json_file:
    json_file.write(d_model_json)

combined_model_json = combined_model.to_json()
with open("./model2_output/save_model/combined_model.json", "w") as json_file:
    json_file.write(combined_model_json)

print("Saved model to disk")

#print("save model ")
plt.figure(1, figsize=(16, 8))
plt.plot(d_loss_real_history)
plt.ylabel('d_loss_real')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./model2_output/image/d_loss_real_history.png')

plt.figure(2, figsize=(16, 8))
plt.plot(d_loss_fake_history)
plt.ylabel('d_loss_fake')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./model2_output/image/d_loss_fake_history.png')

#plt.show()

plt.figure(3, figsize=(16, 8))
plt.plot(g_loss_history)
plt.ylabel('g_loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./model2_output/image/g_loss_history.png')

#plt.show()\

plt.show()

print("Finish")

/root/jupyter/inspace/sang-min/myo_proejct
30
./dataset_2018_05_16/1/
15
Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       1280      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       64        
______________________________________________

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss real: 0.180036] [D loss fake: 0.430595] [D loss: 0.610630] [G loss: 0.224385]
gan imaga2 :  (128, 128, 1)
1 [D loss real: 0.183310] [D loss fake: 0.429223] [D loss: 0.612533] [G loss: 0.197139]
2 [D loss real: 0.244635] [D loss fake: 0.421162] [D loss: 0.665798] [G loss: 0.139581]
3 [D loss real: 0.176632] [D loss fake: 0.403779] [D loss: 0.580411] [G loss: 0.126389]
4 [D loss real: 0.173402] [D loss fake: 0.430998] [D loss: 0.604400] [G loss: 0.174032]
5 [D loss real: 0.164306] [D loss fake: 0.421116] [D loss: 0.585422] [G loss: 0.292463]
6 [D loss real: 0.234437] [D loss fake: 0.442177] [D loss: 0.676614] [G loss: 0.292481]
7 [D loss real: 0.095687] [D loss fake: 0.409683] [D loss: 0.505370] [G loss: 0.166356]
8 [D loss real: 0.278826] [D loss fake: 0.425223] [D loss: 0.704049] [G loss: 0.252334]
9 [D loss real: 0.177202] [D loss fake: 0.431420] [D loss: 0.608622] [G loss: 0.180118]
10 [D loss real: 0.227096] [D loss fake: 0.408729] [D loss: 0.635826] [G loss: 0.141179]
11 

91 [D loss real: 0.318050] [D loss fake: 0.427817] [D loss: 0.745867] [G loss: 0.213132]
92 [D loss real: 0.165244] [D loss fake: 0.419025] [D loss: 0.584269] [G loss: 0.125008]
93 [D loss real: 0.185906] [D loss fake: 0.399614] [D loss: 0.585520] [G loss: 0.191363]
94 [D loss real: 0.229469] [D loss fake: 0.442217] [D loss: 0.671685] [G loss: 0.192719]
95 [D loss real: 0.212507] [D loss fake: 0.479429] [D loss: 0.691936] [G loss: 0.228696]
96 [D loss real: 0.174468] [D loss fake: 0.468082] [D loss: 0.642550] [G loss: 0.229956]
97 [D loss real: 0.260001] [D loss fake: 0.460541] [D loss: 0.720542] [G loss: 0.218527]
./dataset_2018_05_16/8/
98 [D loss real: 0.181096] [D loss fake: 0.443367] [D loss: 0.624463] [G loss: 0.200127]
99 [D loss real: 0.168692] [D loss fake: 0.448947] [D loss: 0.617638] [G loss: 0.166151]
100 [D loss real: 0.228013] [D loss fake: 0.432991] [D loss: 0.661004] [G loss: 0.184076]
101 [D loss real: 0.180702] [D loss fake: 0.464647] [D loss: 0.645349] [G loss: 0.165

181 [D loss real: 0.131883] [D loss fake: 0.385435] [D loss: 0.517318] [G loss: 0.184852]
./dataset_2018_05_16/14/
182 [D loss real: 0.131838] [D loss fake: 0.435501] [D loss: 0.567339] [G loss: 0.244973]
183 [D loss real: 0.237183] [D loss fake: 0.425871] [D loss: 0.663054] [G loss: 0.202555]
184 [D loss real: 0.092964] [D loss fake: 0.411959] [D loss: 0.504924] [G loss: 0.303715]
185 [D loss real: 0.252330] [D loss fake: 0.430939] [D loss: 0.683269] [G loss: 0.183007]
186 [D loss real: 0.165307] [D loss fake: 0.427166] [D loss: 0.592474] [G loss: 0.209956]
187 [D loss real: 0.129940] [D loss fake: 0.451490] [D loss: 0.581430] [G loss: 0.188720]
188 [D loss real: 0.127929] [D loss fake: 0.436070] [D loss: 0.564000] [G loss: 0.187391]
189 [D loss real: 0.240820] [D loss fake: 0.443886] [D loss: 0.684706] [G loss: 0.208182]
190 [D loss real: 0.189711] [D loss fake: 0.452193] [D loss: 0.641904] [G loss: 0.211974]
191 [D loss real: 0.253387] [D loss fake: 0.444337] [D loss: 0.697723] [G l

...

12606 [D loss real: 0.210283] [D loss fake: 0.424274] [D loss: 0.634558] [G loss: 0.143300]
12607 [D loss real: 0.109222] [D loss fake: 0.409152] [D loss: 0.518374] [G loss: 0.086941]
12608 [D loss real: 0.261804] [D loss fake: 0.422443] [D loss: 0.684247] [G loss: 0.168754]
12609 [D loss real: 0.170041] [D loss fake: 0.466355] [D loss: 0.636397] [G loss: 0.204032]
12610 [D loss real: 0.192731] [D loss fake: 0.415290] [D loss: 0.608021] [G loss: 0.262851]
12611 [D loss real: 0.192000] [D loss fake: 0.448847] [D loss: 0.640847] [G loss: 0.085056]
12612 [D loss real: 0.249215] [D loss fake: 0.422043] [D loss: 0.671258] [G loss: 0.152912]
12613 [D loss real: 0.217350] [D loss fake: 0.417487] [D loss: 0.634837] [G loss: 0.123366]
12614 [D loss real: 0.146674] [D loss fake: 0.389587] [D loss: 0.536260] [G loss: 0.183019]
12615 [D loss real: 0.219375] [D loss fake: 0.431503] [D loss: 0.650877] [G loss: 0.256429]
12616 [D loss real: 0.109950] [D loss fake: 0.436161] [D loss: 0.546111] [G loss

12694 [D loss real: 0.222952] [D loss fake: 0.459527] [D loss: 0.682479] [G loss: 0.211919]
12695 [D loss real: 0.277522] [D loss fake: 0.432095] [D loss: 0.709617] [G loss: 0.279541]
12696 [D loss real: 0.247556] [D loss fake: 0.417299] [D loss: 0.664855] [G loss: 0.111462]
12697 [D loss real: 0.148055] [D loss fake: 0.412245] [D loss: 0.560300] [G loss: 0.224990]
12698 [D loss real: 0.110812] [D loss fake: 0.401818] [D loss: 0.512630] [G loss: 0.221722]
12699 [D loss real: 0.214368] [D loss fake: 0.426642] [D loss: 0.641010] [G loss: 0.105666]
12700 [D loss real: 0.070472] [D loss fake: 0.438306] [D loss: 0.508777] [G loss: 0.283250]
12701 [D loss real: 0.157286] [D loss fake: 0.436488] [D loss: 0.593774] [G loss: 0.182515]
./dataset_2018_05_16/5/
12702 [D loss real: 0.187976] [D loss fake: 0.426650] [D loss: 0.614626] [G loss: 0.158572]
12703 [D loss real: 0.170220] [D loss fake: 0.425679] [D loss: 0.595898] [G loss: 0.177430]
12704 [D loss real: 0.186402] [D loss fake: 0.447838] [D

12782 [D loss real: 0.230558] [D loss fake: 0.408127] [D loss: 0.638685] [G loss: 0.202651]
12783 [D loss real: 0.213007] [D loss fake: 0.405562] [D loss: 0.618569] [G loss: 0.194545]
12784 [D loss real: 0.199569] [D loss fake: 0.446243] [D loss: 0.645811] [G loss: 0.169246]
12785 [D loss real: 0.247635] [D loss fake: 0.449038] [D loss: 0.696673] [G loss: 0.089085]
12786 [D loss real: 0.164168] [D loss fake: 0.457036] [D loss: 0.621204] [G loss: 0.201868]
./dataset_2018_05_16/11/
12787 [D loss real: 0.167964] [D loss fake: 0.429898] [D loss: 0.597862] [G loss: 0.157364]
12788 [D loss real: 0.247021] [D loss fake: 0.432191] [D loss: 0.679213] [G loss: 0.197910]
12789 [D loss real: 0.177043] [D loss fake: 0.437447] [D loss: 0.614490] [G loss: 0.210370]
12790 [D loss real: 0.226646] [D loss fake: 0.414005] [D loss: 0.640651] [G loss: 0.220160]
12791 [D loss real: 0.232025] [D loss fake: 0.424379] [D loss: 0.656404] [G loss: 0.178494]
12792 [D loss real: 0.194759] [D loss fake: 0.377713] [

12870 [D loss real: 0.095944] [D loss fake: 0.398840] [D loss: 0.494784] [G loss: 0.156377]
./dataset_2018_05_16/2/
12871 [D loss real: 0.188079] [D loss fake: 0.446384] [D loss: 0.634462] [G loss: 0.331983]
12872 [D loss real: 0.131650] [D loss fake: 0.414341] [D loss: 0.545991] [G loss: 0.213209]
12873 [D loss real: 0.138053] [D loss fake: 0.419627] [D loss: 0.557680] [G loss: 0.184990]
12874 [D loss real: 0.167114] [D loss fake: 0.417682] [D loss: 0.584796] [G loss: 0.253081]
12875 [D loss real: 0.254206] [D loss fake: 0.420155] [D loss: 0.674361] [G loss: 0.149998]
12876 [D loss real: 0.129096] [D loss fake: 0.428347] [D loss: 0.557443] [G loss: 0.200450]
12877 [D loss real: 0.136140] [D loss fake: 0.418193] [D loss: 0.554333] [G loss: 0.233698]
12878 [D loss real: 0.147340] [D loss fake: 0.398296] [D loss: 0.545636] [G loss: 0.082487]
12879 [D loss real: 0.179136] [D loss fake: 0.401263] [D loss: 0.580399] [G loss: 0.232426]
12880 [D loss real: 0.148865] [D loss fake: 0.452548] [D

12958 [D loss real: 0.126318] [D loss fake: 0.470628] [D loss: 0.596947] [G loss: 0.251977]
12959 [D loss real: 0.245889] [D loss fake: 0.477436] [D loss: 0.723325] [G loss: 0.213478]
12960 [D loss real: 0.193644] [D loss fake: 0.416289] [D loss: 0.609933] [G loss: 0.268289]
12961 [D loss real: 0.146184] [D loss fake: 0.404760] [D loss: 0.550944] [G loss: 0.201216]
12962 [D loss real: 0.178733] [D loss fake: 0.448834] [D loss: 0.627567] [G loss: 0.243947]
12963 [D loss real: 0.186062] [D loss fake: 0.424779] [D loss: 0.610841] [G loss: 0.126141]
12964 [D loss real: 0.114348] [D loss fake: 0.419299] [D loss: 0.533647] [G loss: 0.243059]
12965 [D loss real: 0.231639] [D loss fake: 0.407259] [D loss: 0.638898] [G loss: 0.222331]
12966 [D loss real: 0.216544] [D loss fake: 0.418954] [D loss: 0.635498] [G loss: 0.249416]
12967 [D loss real: 0.244601] [D loss fake: 0.418114] [D loss: 0.662714] [G loss: 0.198851]
12968 [D loss real: 0.180970] [D loss fake: 0.427034] [D loss: 0.608004] [G loss